In [1]:
'''
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    test/
        test/
            001,jpg
```
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-8332 in data/train/cats
- put the cat pictures index 8333-12499 in data/validation/cats
- put the dogs pictures index 0-8332 in data/train/dogs
- put the dog pictures index 8333-12499 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:

'''
import os,shutil
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# 设置参数，文件夹位置，建议先把epochs调成1来测试模型完整。如果不用gpu加速可以调小batch_size
img_width, img_height = 150, 150

train_data_dir = 'final_dogs&cats\\data\\train'
validation_data_dir = '\final_dogs&cats\\data\\validation'
predict_data_dir = 'final_dogs&cats\\data\\test'
path_folder='final_dogs&cats\\'

nb_train_samples = len(os.listdir(train_data_dir+os.sep+'dogs'))*2
nb_validation_samples = len(os.listdir(validation_data_dir+os.sep+'dogs'))*2
#第一次测试，调整为epochs=1，测试完整性
epochs = 50
batch_size = 512

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
print('gpu imported')

Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: 'final_dogs&cats\\data\\train/dogs'

In [99]:
#设置图像生成器
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

predict_generator = test_datagen.flow_from_directory(
    predict_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,)

print("Image prepared")

Found 16666 images belonging to 2 classes.
Found 8334 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Image prepared


In [100]:
#基础模型
print('Training start')
import time
start_time=time.time()

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
end_time=time.time()
print('Running time', end_time-start_time)
model.save_weights(path_folder+'first_try.h5')
print('Model Saved to',path_folder+'first_try.h5')

Training start
Epoch 1/50
32/32 [==============================] - 84s 3s/step - loss: 0.7831 - acc: 0.5390 - val_loss: 0.6652 - val_acc: 0.6387
Epoch 2/50
32/32 [==============================] - 82s 3s/step - loss: 0.6648 - acc: 0.6090 - val_loss: 0.6501 - val_acc: 0.5994
Epoch 3/50
32/32 [==============================] - 80s 3s/step - loss: 0.6335 - acc: 0.6491 - val_loss: 0.6269 - val_acc: 0.6180
Epoch 4/50
32/32 [==============================] - 79s 2s/step - loss: 0.5886 - acc: 0.6849 - val_loss: 0.7683 - val_acc: 0.5752
Epoch 5/50
32/32 [==============================] - 76s 2s/step - loss: 0.5837 - acc: 0.6941 - val_loss: 0.5194 - val_acc: 0.7474
Epoch 6/50
32/32 [==============================] - 74s 2s/step - loss: 0.5484 - acc: 0.7208 - val_loss: 0.5186 - val_acc: 0.7393
Epoch 7/50
32/32 [==============================] - 73s 2s/step - loss: 0.5428 - acc: 0.7201 - val_loss: 0.5011 - val_acc: 0.7535
Epoch 8/50
32/32 [==============================] - 70s 2s/step - loss: 0.5

In [2]:
p=model.predict_generator(predict_generator)
test_img = os.listdir(predict_data_dir+os.sep+'test')
label=[]
for i in p:
    if i>0.5:
        label.append('cat')
    else:
        label.append('dot')
label

import pandas as pd
prediction=pd.DataFrame([test_img,label]).T

#col_name=['Images']+['Label']
prediction.to_csv(path_folder+'prediction.csv',header=True,index=False)
print('Prediction save to ',path_folder)

NameError: name 'model' is not defined

In [102]:
#查看每次的修改过的模型结构，将模型结构，准确率记录到log中，方便最后整理report。
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_31 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_32 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 34, 34, 64)        18496     
__________